Add Lib

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Add and Remove Data NULL

In [2]:
file_path = '/content/Data_With_Label.csv'
data = pd.read_csv(file_path,encoding = 'utf-8-sig')
temp = data.loc[(data['Quanlity'] == 0) & (data['Price'] == 0) & (data['Packaging'] == 0) & (data['Service'] == 0)].index
data = data.drop(temp)

Split Sen and Label

In [3]:
X_sen = data.sentences.values
Y = data[['Quanlity','Price','Packaging','Service']].values

Add Stop Word

In [4]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

# Remove Word have NOT
blabla = ['weren',"weren't",'doesn','hasn',"hasn't",'haven',"haven't","isn't",
          "won't",'wouldn',"wouldn't",
          "aren't", "couldn't",'couldn', 'didn', "didn't",'doesn',"doesn't",'don',"don't",'hadn',"hadn't",
          'isn',"isn't",'mightn',"mightn't",'mustn',"mustn't",'needn',"needn't",'shan',"shan't",'shouldn',
          "shouldn't",'wasn',"wasn't", "didn't",'not', 'nor','aren']
for word in blabla:
  if(word in stop_words):
    stop_words.remove(word)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# CLean And Pre-processing Data

In [5]:
from nltk.stem import LancasterStemmer
wnl = WordNetLemmatizer()
stemmer = LancasterStemmer()

# Word #

word_dict = {"n't" : " not","'re" : " are","'s" : " is", "'ve" : " have"}
def replaceWord(Word):
  w = ''
  run = False
  for word in Word:
    if word == "'":
      run = True
  if run:
    for word in word_dict:
      Word = Word.replace(word,word_dict[word])
  return Word

# Sentences #

def replaceDot(Sen):
  w = ''
  for word in Sen:
    if (ord(word) in range(33,48)) or (ord(word) in range(58,65)) or (ord(word) in range(91,97)):
      if word == "'":
        w += word
      else:
        w += '.'
    else:
      w += word.lower()
  return replaceWord(w)

Links = ['and','but','howerver']
def splitSen(Sen):
  temp = Sen.split('.')
  temp_2 = []
  for sen in temp:
    s = ''
    for word in sen.split():
      if word in Links:
        temp_2.append(s[:-1])
        s = ''
      else:
        s += word + ' '
    if s != '':
      temp_2.append(s[:-1])

  return temp_2

def removeStopWord(Sen):
  s = ''
  for word in Sen.split():
    if word not in stop_words:
      s += stemmer.stem(word) + ' '

  return s[:-1]

# Main Pre-processing Sen #

def cleanSen(Sen):
  Sen = replaceDot(Sen)
  temp = splitSen(Sen)
  for index in range(len(temp)):
    temp[index] = removeStopWord(temp[index])

  return temp

clean sen

In [6]:
for index in range(len(X_sen)):
  try:
    X_sen[index] =  removeStopWord(X_sen[index])
  except:
    print(index)

create Key_df

In [7]:
col_names = ['Key','Amount_appear','Positive','Negative','Quality','Price','Packaging','Service']
Keys = pd.DataFrame(columns = col_names)
Keys[col_names[1:]] = Keys[col_names[1:]].astype(float)
index_sen = 0

for sen in X_sen:
  for Word in sen.split():
    if Keys.loc[Keys['Key'] == Word].empty == False:
      Keys.loc[Keys['Key'] == Word,'Amount_appear'] += 1
      Keys.loc[Keys['Key'] == Word,col_names[-4:]] += abs(np.array(Y[index_sen]).reshape(1,4))
      for item in Y[index_sen]:
        if item == 1:
          Keys.loc[Keys['Key'] == Word,'Positive'] += 1
        elif item == -1:
          Keys.loc[Keys['Key'] == Word,'Negative'] += 1
    else:
      temp = [Word,1,0,0,abs(Y[index_sen,0]),abs(Y[index_sen,1]),abs(Y[index_sen,2]),abs(Y[index_sen,3])]
      for item in Y[index_sen]:
        if item == 1:
          temp[1] += 1
        elif item == -1:
          temp[2] += 1
      temp = pd.DataFrame(np.array(temp).reshape(1,8),columns = col_names)
      temp[col_names[1:]] = temp[col_names[1:]].astype(float)
      Keys = pd.concat([Keys,temp],axis = 0)
  index_sen += 1

Make Weight

In [8]:
def makeWeight(vec):
  if vec[1] * vec[2] == 0:
    if vec[1] >= vec[0]:
      return [vec[0],1,0]
    elif vec[2] >= vec[0]:
      return [vec[0],0,1]
    return [vec[0],0,0]

  if vec[1] / vec[2] >= 2:
      return [vec[0],int(vec[1] / vec[2]),0]
  if vec[2] / vec[1] >= 2:
    return [vec[0],0,int(vec[2] / vec[1])]

  if vec[1] + vec[2] < vec[0]:
    return [vec[0],0,0]
  return [vec[0],0,0]

Keys_2 = Keys.copy()
for index in range(Keys.shape[0]):
  Keys_2.iloc[index,1:4] = np.array(makeWeight(Keys_2.iloc[index,1:4].values)).reshape(1,3)
  temp = Keys_2.iloc[index,-4:].sum()
  if temp != 0:
    Keys_2.iloc[index,-4:] /= (temp)

Scaler

In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 5))

Keys_2[col_names[2:4]]= scaler.fit_transform(Keys_2[col_names[2:4]])
Keys_2.describe()

,Amount_appear,Positive,Negative,Quality,Price,Packaging,Service
count,1720.000000,1720.000000,1720.000000,1720.000000,1720.000000,1720.000000,1720.000000
mean,3.583721,0.134884,0.052437,0.401333,0.172629,0.242054,0.183984
std,6.468429,0.269614,0.270581,0.440341,0.330168,0.358603,0.313729
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000
75%,3.000000,0.250000,0.000000,1.000000,0.166667,0.500000,0.333333
max,91.000000,5.000000,5.000000,1.000000,1.000000,1.000000,1.000000


Sen to Vector

In [10]:
def sen2vec(Sen):
  temp = pd.DataFrame(columns = col_names)
  for word in Sen.split():
    if Keys_2.loc[Keys_2['Key'] == word].empty == False:
      temp = pd.concat([temp,Keys_2.loc[Keys_2['Key'] == word]], axis = 0)

  if temp.empty:
    vec = [0,0,0,0,0,0]
    return np.array(vec).reshape(1,6)
  return temp.iloc[:,2:].sum().values.reshape(1,6)

# Main #

X = sen2vec(X_sen[0])
for index in range(1,len(X_sen)):
  X = np.vstack((X,sen2vec(X_sen[index])))

process Label

In [11]:
def processLabel(Label):
  a = 1
  for item in Label:
    if item != 0:
      a *= item
  if a == 1:
    return np.append([1,0],abs(Label))
  if a == -1:
    return np.append([0,1],abs(Label))

Y_pro = processLabel(Y[0])
for index in range(1,len(Y)):
  Y_pro = np.vstack((Y_pro,processLabel(Y[index])))

Make Train and Test

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y_pro, test_size= 0.2, random_state = 42)

Model

In [13]:
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

md_pos = svm.SVC(kernel = 'rbf', C = 100,gamma = 0.1)
md_neg = svm.SVC(kernel = 'rbf',C = 100,gamma = 0.1)
md_qua = svm.SVC(kernel = 'rbf')
md_pri = svm.SVC(kernel = 'rbf')
md_pac = svm.SVC(kernel = 'rbf')
md_ser = svm.SVC(kernel = 'rbf')


md_pos.fit(X_train[:,:2].reshape(-1,2),y_train[:,0])
md_neg.fit(X_train[:,:2].reshape(-1,2),y_train[:,1])
md_qua.fit(X_train[:,2:].reshape(-1,4),y_train[:,2])
md_pri.fit(X_train[:,2:].reshape(-1,4),y_train[:,3])
md_pac.fit(X_train[:,2:].reshape(-1,4),y_train[:,4])
md_ser.fit(X_train[:,2:].reshape(-1,4),y_train[:,5])

def md_predict(vec):
  lb = [0,0,0,0,0,0]
  lb[0] = md_pos.predict(vec[:2].reshape(1,2))[0]
  lb[1] = md_neg.predict(vec[:2].reshape(1,2))[0]
  lb[2] = md_qua.predict(vec[2:].reshape(1,4))[0]
  lb[3] = md_pri.predict(vec[2:].reshape(1,4))[0]
  lb[4] = md_pac.predict(vec[2:].reshape(1,4))[0]
  lb[5] = md_ser.predict(vec[2:].reshape(1,4))[0]
  return np.array(lb)

y_pred = md_predict(X_test[0])
for index in range(1,len(X_test)):
  y_pred = np.vstack([y_pred,md_predict(X_test[index])])

Show Accuracy

In [14]:
from sklearn.metrics import classification_report

for i in range(y_test.shape[1]):
  print(classification_report(y_test[:, i], y_pred[:, i], digits=4))

              precision    recall  f1-score   support

           0     0.6250    0.7377    0.6767        61
           1     0.8242    0.7353    0.7772       102

    accuracy                         0.7362       163
   macro avg     0.7246    0.7365    0.7269       163
weighted avg     0.7496    0.7362    0.7396       163

              precision    recall  f1-score   support

           0     0.8242    0.7353    0.7772       102
           1     0.6250    0.7377    0.6767        61

    accuracy                         0.7362       163
   macro avg     0.7246    0.7365    0.7269       163
weighted avg     0.7496    0.7362    0.7396       163

              precision    recall  f1-score   support

           0     0.9815    0.9725    0.9770       109
           1     0.9455    0.9630    0.9541        54

    accuracy                         0.9693       163
   macro avg     0.9635    0.9677    0.9655       163
weighted avg     0.9695    0.9693    0.9694       163

              preci

In [15]:
for index in range(6):
  x = [[0,0],[0,0]]
  for index_2 in range(len(y_test)):
    x[y_test[index_2][index]][y_pred[index_2][index]] += 1
  print(col_names[2 + index])
  print(x)

Positive
[[45, 16], [27, 75]]
Negative
[[75, 27], [16, 45]]
Quality
[[106, 3], [2, 52]]
Price
[[133, 0], [3, 27]]
Packaging
[[115, 1], [4, 43]]
Service
[[110, 1], [5, 47]]


# Predict Review

In [66]:
def analReview(Review):
  sen_list = cleanSen(Review)
  pos = np.array([0,0,0,0]).reshape(1,4)
  neg = np.array([0,0,0,0]).reshape(1,4)

  vec_list = []
  for sen in sen_list:
    vec_list.append(sen2vec(sen)[0])

  for index in range(len(vec_list)):
    temp = md_predict(vec_list[index])
    if temp[0] == 1:
      pos += temp[2:]
    if temp[1] == 1:
      neg += temp[2:]

  sen_dict = {
      0 : 'Quality',
      1 : 'Price',
      2 : 'Packaging',
      3 : 'Service'
  }

  for index in range(4):
    result = 'None'
    if pos[0,index] == 1:
      result = 'pos'
    if neg[0,index] == 1:
      if result == 'None':
        result = 'neg'
      else:
        result += ' - neg'
    print(sen_dict[index]," : ",result)


  return pos, neg

text = "The price was great, but the taste is not great. The expiration date is 2023. It’s flat."
analReview(text)

Quality  :  pos
Price  :  pos
Packaging  :  None
Service  :  pos


(array([[1, 1, 0, 1]]), array([[0, 0, 0, 0]]))